In [12]:
# import libraries
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re
import pandas as pd
nltk.download('stopwords')
from nltk.corpus import stopwords
from textstat import syllable_count
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Divine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Divine\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Divine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
df = pd.read_excel("C:/Users/Divine/Downloads/Output Data Structure.xlsx")
def process_link(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    x = soup.findAll(attrs= {'class': 'td-post-title'})
    title_text = x[0].text
    first_newline_index = title_text.find("\n")
    second_newline_index = title_text.find("\n", first_newline_index + 1)

    line_between_newlines = title_text[first_newline_index + 1 : second_newline_index]

    title = line_between_newlines
    y = soup.findAll(attrs = {'class': 'td-post-content tagdiv-type'})
    content = y[0].text.strip()+" "+title
    all_stopwords_given = []
    with open("C:/Users/Divine/Downloads/Stopwords.csv.csv", newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            for item in row:
                all_stopwords_given.append(item)
    positive_words = []

    with open("C:/Users/Divine/Downloads/positive-words.csv", newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            for item in row:
                positive_words.append(item)
    positive_words_final = []
    for i in positive_words:
        if i not in all_stopwords_given:
            positive_words_final.append(i)
    negative_words = []
    with open("C:/Users/Divine/Downloads/negative-words.csv", newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            for item in row:
                negative_words.append(item)
    negative_words_final = []
    for i in negative_words:
        if i not in all_stopwords_given:
            negative_words_final.append(i)
    stop_words = stopwords.words('english')
    all_stopwords = all_stopwords_given+stop_words
    delimiters = [',', ';', ':', '.', '?', '!', '-', '_', '/', '\\', '|', '&', '+', '*', '(', ')', '[', ']', '{', '}']
    token_text = nltk.word_tokenize(content)
    for i in token_text:
        if i in all_stopwords or i in delimiters:
            token_text.remove(i)
    positive_score = 0
    for i in token_text:
        if i in positive_words_final:
            positive_score = positive_score+1

    negative_score = 0
    for i in token_text:
        if i in negative_words_final:
            negative_score = negative_score+1

    Polarity_Score = (positive_score-negative_score)/ ((positive_score + negative_score) + 0.000001)
    
    Subjectivity_Score = (positive_score + negative_score)/ ((len(token_text)) + 0.000001)
    
    sentences = nltk.sent_tokenize(content)
    num_sentences = len(sentences)
    Average_Sentence_Length = len(token_text) / num_sentences
    c= 0
    for word in token_text:
        if syllable_count(word) > 2 and word.endswith('es')==False and word.endswith('ed')==False:
            c = c+1
    Percentage_of_Complex_words = c / len(token_text)
    Fog_Index = 0.4 * (Average_Sentence_Length +Percentage_of_Complex_words)
    Average_Number_of_Words_Per_Sentence = len(token_text) / num_sentences
    pattern = r"\b(I|we|my|ours|us)\b"
    count = len(re.findall(pattern, content, flags=re.IGNORECASE))
    total_chars = sum(len(word) for word in token_text)
    Average_Word_Length = total_chars/len(token_text)
    def count_syllables_per_word(text):
        words = text.split()
        total_syllable_count = 0
        for word in words:
            num_vowels = 0
            prev_char_was_vowel = False
            for char in word:
                char = char.lower()
                if char in "aeiouy":
                    if not prev_char_was_vowel:
                        num_vowels += 1
                    prev_char_was_vowel = True
                else:
                    prev_char_was_vowel = False
            if word.endswith(("es", "ed")) and num_vowels >= 1:
                num_vowels -= 1
            total_syllable_count += num_vowels
        return total_syllable_count
    total_syllable_count = count_syllables_per_word(content)
    #syllable count per word
    avg_syllable_count_per_word = total_syllable_count / len(token_text)
    return {
        'link': link,
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': Polarity_Score,
        'SUBJECTIVITY SCORE': Subjectivity_Score,
        'AVG SENTENCE LENGTH': Average_Sentence_Length,
        'PERCENTAGE OF COMPLEX WORDS': Percentage_of_Complex_words,
        'FOG INDEX': Fog_Index,
        'AVG NUMBER OF WORDS PER SENTENCE': Average_Number_of_Words_Per_Sentence,
        'COMPLEX WORD COUNT': c,
        'WORD COUNT': len(token_text),
        'SYLLABLE PER WORD': avg_syllable_count_per_word,
        'PERSONAL PRONOUNS': count,
        'AVG WORD LENGTH': Average_Word_Length
    }

In [49]:
output_data = []
z = 0
for link in df['URL']:
    output_data.append(process_link(link))
print(output_data)

[{'link': 'https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/', 'POSITIVE SCORE': 63, 'NEGATIVE SCORE': 33, 'POLARITY SCORE': 0.31249999674479173, 'SUBJECTIVITY SCORE': 0.07289293844123543, 'AVG SENTENCE LENGTH': 17.103896103896105, 'PERCENTAGE OF COMPLEX WORDS': 0.24069855732725892, 'FOG INDEX': 6.937837864489346, 'AVG NUMBER OF WORDS PER SENTENCE': 17.103896103896105, 'COMPLEX WORD COUNT': 317, 'WORD COUNT': 1317, 'SYLLABLE PER WORD': 2.6385725132877753, 'PERSONAL PRONOUNS': 1, 'AVG WORD LENGTH': 6.5596051632498105}, {'link': 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/', 'POSITIVE SCORE': 56, 'NEGATIVE SCORE': 38, 'POLARITY SCORE': 0.19148935966500683, 'SUBJECTIVITY SCORE': 0.09428284845107036, 'AVG SENTENCE LENGTH': 12.308641975308642, 'PERCENTAGE OF COMPLEX WORDS': 0.1534603811434303, 'FOG INDEX': 4.9848409425808295, 'AVG NUMBER OF WORDS PER SENTENCE': 12.308641975308642, 'COMPLEX WORD COUNT': 153, 'WORD COUNT': 997

In [51]:
output_df = pd.DataFrame(columns=['link', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])
for data in output_data:
    output_df = output_df.append(data, ignore_index=True)
print(output_df)

C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(data, ignore_index=True)
C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(data, ignore_index=True)
C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(data, ignore_index=True)
C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.appe

                                                 link POSITIVE SCORE  \
0   https://insights.blackcoffer.com/ai-in-healthc...             63   
1   https://insights.blackcoffer.com/what-if-the-c...             56   
2   https://insights.blackcoffer.com/what-jobs-wil...             66   
3   https://insights.blackcoffer.com/will-machine-...             57   
4   https://insights.blackcoffer.com/will-ai-repla...             50   
5   https://insights.blackcoffer.com/man-and-machi...             44   
6   https://insights.blackcoffer.com/in-future-or-...             26   
7   https://insights.blackcoffer.com/how-machine-l...             36   
8   https://insights.blackcoffer.com/deep-learning...             64   
9   https://insights.blackcoffer.com/how-to-protec...             46   
10  https://insights.blackcoffer.com/how-machines-...             35   
11  https://insights.blackcoffer.com/ai-human-robo...             27   
12  https://insights.blackcoffer.com/how-ai-will-c...           

C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(data, ignore_index=True)
C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(data, ignore_index=True)
C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(data, ignore_index=True)
C:\Users\Divine\AppData\Local\Temp\ipykernel_17080\3234180614.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.appe

In [56]:
output_df.to_csv('output.csv', index=False)